In [109]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from torch.nn import functional as F
import transformers
import re

# generator = pipeline('text-generation', model='flax-community/papuGaPT2', device=0)
# model_name = 'flax-community/papuGaPT2'
generator = pipeline('text-generation', model='eryk-mazus/polka-1.1b-chat', device=0)
model_name = 'eryk-mazus/polka-1.1b-chat'
transformers.logging.set_verbosity_error()
device = 'cuda'
device = 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label
    
            
def sentence_prob(sentence_txt):
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(device)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()

def normalized_sentence_prob(txt):
    length = len(tokenizer(txt, return_tensors='pt')['input_ids'][0])
    return sentence_prob(txt) / length

In [95]:
def beam_search(k, sentence):
    words = sentence.split(' ')
    beams = []

    beams = [(normalized_sentence_prob(w), w) for w in words[0].split('|')]
    beams.sort(key=lambda x: x[0], reverse=True)
    print(beams)
    # beams = [w[1] for w in beams]
    beams = beams[:k]
    
    for i in words[1:]:
        arr = [(normalized_sentence_prob(b[1] + ' ' + w), b[1] + ' ' + w) for b in beams for w in i.split('|')]
        arr.sort(key=lambda x:x[0], reverse=True)
        print(arr)
        # arr = [w[1] for w in arr]
        arr = arr[:k]
        print(arr)
        beams = arr
    
    return beams



In [107]:
input1 = 'wprost|wyprosty|wyprostu|wyprost uwielbiała|wielbił|wielbiła|uwielbił|wielbiło|uwielbiał|uwielbiało|uwielbiały słuchać|osłuchać|słychać|usłuchać o|i|e|a|ó|ę|y|ą|u wartościach własnych|owłosionych macierzy|mocarz|macierzą|macierze|mocarza|mocarze|mocarzy|macierz'
input2 = 'ala|marianna ma|miała rudego|siwego|białego kota|psa'
input3 = 'wartościach własnych|owłosionych macierzy|mocarz|macierzą|macierze|mocarza|mocarze|mocarzy|macierz'


In [112]:
print(normalized_sentence_prob('wyprosty uwielbiały słuchać o wartościach własnych macierzy'))
print(normalized_sentence_prob('wprost uwielbiał słuchać o wartościach własnych macierzy'))

-3.629666283017113
-4.113057989823191


In [111]:
beam_search(2, input1)


[(-5.443730354309082, 'wyprosty'), (-5.769036293029785, 'wyprostu'), (-5.851399103800456, 'wyprost'), (-6.551085154215495, 'wprost')]
[(-4.737009525299072, 'wyprosty uwielbiały'), (-4.914733409881592, 'wyprosty uwielbiała'), (-4.923218727111816, 'wyprostu uwielbiała'), (-5.054885387420654, 'wyprosty uwielbiało'), (-5.061822891235352, 'wyprostu uwielbiało'), (-5.1802215576171875, 'wyprostu uwielbiały'), (-5.323688507080078, 'wyprosty uwielbiał'), (-5.5633746555873325, 'wyprostu uwielbiał'), (-5.836857386997768, 'wyprosty wielbił'), (-5.944157191685268, 'wyprostu wielbił'), (-5.964850289481027, 'wyprosty wielbiła'), (-6.078267778669085, 'wyprosty wielbiło'), (-6.182826450892857, 'wyprostu wielbiła'), (-6.288023267473493, 'wyprostu wielbiło'), (-7.497186388288226, 'wyprosty uwielbił'), (-7.6839174543108255, 'wyprostu uwielbił')]
[(-4.737009525299072, 'wyprosty uwielbiały'), (-4.914733409881592, 'wyprosty uwielbiała')]
[(-4.078137484463778, 'wyprosty uwielbiały słuchać'), (-4.2068217884410

[(-3.629666283017113,
  'wyprosty uwielbiały słuchać o wartościach własnych macierzy'),
 (-3.7504984537760415,
  'wyprosty uwielbiały słuchać o wartościach własnych macierz')]